In [31]:
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
from textblob import TextBlob
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import openai

# WebScrap the Articles for its headline, summary and link and sending to email

In [ ]:
# Email Configuration
# Email Configuration
SMTP_SERVER = "smtp.gmail.com"  # For Gmail
SMTP_PORT = 587
EMAIL_SENDER = "Enter your email"
EMAIL_PASSWORD = "Put in your own email code"  # Use an App Password for Gmail
EMAIL_RECEIVER = "Enter reciever email"
# Rate limiting parameters
MAX_TOKENS_PER_MINUTE = 40000  # 40,000 tokens per minute in the free tier
MAX_REQUESTS_PER_MINUTE = 3  # 3 requests per minute in the free tier
MAX_REQUESTS_PER_DAY = 200  # 200 requests per day in the free tier

tokens_used = 0
requests_made = 0

def fetch_news():
    url = "https://www.bbc.com/business"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    for item in soup.select('[data-testid="internal-link"]'):
        title_tag = item.select_one('[data-testid="card-headline"]')
        
        if title_tag:  # Ensure the title exists
            title = title_tag.text.strip()
            link = item.get('href')
            if link.startswith("/"):
                link = "https://www.bbc.com" + link

            # Fetch the article content
            article_response = requests.get(link)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # Extract all paragraphs from the article
            paragraphs = article_soup.find_all('p')
            article_text = " ".join([p.get_text() for p in paragraphs])
            
            articles.append({"title": title, "link": link, "summary": article_text})


    return pd.DataFrame(articles)

def send_email(news_df):
    # Format email content
    email_body = "<h2>Latest BBC Business News</h2>"
    for _, row in news_df.iterrows():
        email_body += f"<h3>{row['title']}</h3>"
        email_body += f"<p>{row['summary']}</p>"
        email_body += f'<p><a href="{row["link"]}">Read Full Article</a></p><hr>'
    
    # Set up email
    msg = MIMEMultipart()
    msg['From'] = EMAIL_SENDER
    msg['To'] = EMAIL_RECEIVER
    msg['Subject'] = "Daily BBC Business News Summary"
    msg.attach(MIMEText(email_body, 'html'))

    # Send email
    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")

# Fetch news and send via email
news_df = fetch_news()
news_df.head()
#send_email(news_df)


,title,link,summary
0,Bubble tea chain bigger than Starbucks sees sh...,https://www.bbc.com/news/articles/c625n63epyzo,Mixue Ice Cream and Tea may be unfamiliar to m...
1,Ban on India's stock market 'She-Wolf' puts re...,https://www.bbc.com/news/articles/cddyye3zplno,"YouTuber Asmita Patel's mission was to ""make I..."
2,Crypto prices rally after Trump backs five coi...,https://www.bbc.com/news/articles/cn0jgggd7r4o,US President Donald Trump has revealed the nam...
3,"In the US, DEI is under attack. But under a di...",https://www.bbc.com/news/articles/c24110m30ddo,"In Union County, South Carolina, the sprawling..."
4,Ban on India's stock market 'She-Wolf' puts re...,https://www.bbc.com/news/articles/cddyye3zplno,"YouTuber Asmita Patel's mission was to ""make I..."


In [5]:
def fetch_news():
    url = "https://www.bbc.com/business"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    for item in soup.select('[data-testid="internal-link"]'):
        title_tag = item.select_one('[data-testid="card-headline"]')
        
        if title_tag:  # Ensure the title exists
            title = title_tag.text.strip()
            link = item.get('href')
            if link.startswith("/"):
                link = "https://www.bbc.com" + link

            # Fetch the article content
            article_response = requests.get(link)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # Extract all paragraphs from the article
            paragraphs = article_soup.find_all('p')
            article_text = " ".join([p.get_text() for p in paragraphs])
            
            articles.append({"title": title, "link": link, "summary": article_text})
            
            break  # Stop after processing the first article

    return pd.DataFrame(articles)

news_df = fetch_news()
news_df.head()


,title,link,summary
0,Bubble tea chain bigger than Starbucks sees sh...,https://www.bbc.com/news/articles/c625n63epyzo,Mixue Ice Cream and Tea may be unfamiliar to m...


# Created a summary using textrank and sent via email 

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from summa.summarizer import summarize
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Email Configuration
SMTP_SERVER = "smtp.gmail.com"  # For Gmail
SMTP_PORT = 587
EMAIL_SENDER = "Enter your email"
EMAIL_PASSWORD = "Put in your own email code"  # Use an App Password for Gmail
EMAIL_RECEIVER = "Enter reciever email"

def fetch_news():
    url = "https://www.bbc.com/business"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    # html class of the news article item group
    for item in soup.select('[data-testid="internal-link"]'):
        # html class fro the title of the article
        title_tag = item.select_one('[data-testid="card-headline"]')
        
        if title_tag:  # Ensure the title exists
            title = title_tag.text.strip()
            link = item.get('href')
            if link.startswith("/"):
                link = "https://www.bbc.com" + link

            # Fetch the article content
            article_response = requests.get(link)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # Extract all paragraphs from the article
            paragraphs = article_soup.find_all('p')
            article_text = " ".join([p.get_text() for p in paragraphs])

            # Generate a summary for the extracted text using TextRank
            summary = generate_summary_with_textrank(article_text)
            
            articles.append({"title": title, "link": link, "summary": summary})

    return pd.DataFrame(articles)

def generate_summary_with_textrank(text):
    try:
        # Using summa's TextRank for summarization
        summary = summarize(text, ratio=0.2)  # Adjust ratio for desired length of summary
        if not summary:
            summary = "Summary could not be generated."
        return summary
    except Exception as e:
        print(f"Error generating summary with TextRank: {e}")
        return "No summary available"

def send_email(news_df):
    # Format email content
    email_body = "<h2>Latest BBC Business News</h2>"
    for _, row in news_df.iterrows():
        email_body += f"<h3>{row['title']}</h3>"
        email_body += f"<p>{row['summary']}</p>"
        email_body += f'<p><a href="{row["link"]}">Read Full Article</a></p><hr>'
    
    # Set up email
    msg = MIMEMultipart()
    msg['From'] = EMAIL_SENDER
    msg['To'] = EMAIL_RECEIVER
    msg['Subject'] = "Daily BBC Business News Summary"
    msg.attach(MIMEText(email_body, 'html'))

    # Send email
    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")

# Fetch news and send via email
news_df = fetch_news()
send_email(news_df)


Email sent successfully!


# Using Openai API for text summarisation example

In [ ]:
import openai

openai.api_key = "Please use your own Api key"

def summarize_text(text, model="gpt-4o-mini"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that summarizes text concisely."},
            {"role": "user", "content": f"Summarize this text:\n{text}"}
        ],
        temperature=0.5
    )
    return response["choices"][0]["message"]["content"]

# Example usage
long_text = """OpenAI has developed ChatGPT, a large language model that can assist users with various tasks, including answering questions, generating text, and summarizing content. 
It is based on the GPT architecture and uses deep learning techniques to understand and generate human-like responses. Many users leverage it for writing assistance, brainstorming, 
and even coding support. As AI models continue to improve, they become more useful in everyday applications."""

summary = summarize_text(long_text)
print(summary)


OpenAI's ChatGPT is a large language model that assists users with tasks like answering questions, generating text, and summarizing content. Utilizing the GPT architecture and deep learning, it produces human-like responses. Users commonly use it for writing, brainstorming, and coding support, and its utility in everyday applications is increasing as AI models improve.


# Summarising Web Scrapped News article ,using Openai API, and sending it via email

In [ ]:
openai.api_key = "Please use your own openai api key"

# Email Configuration
SMTP_SERVER = "smtp.gmail.com"  # For Gmail
SMTP_PORT = 587
EMAIL_SENDER = "Enter your email"
EMAIL_PASSWORD = "Put in your own email code"  # Use an App Password for Gmail
EMAIL_RECEIVER = "Enter reciever email"

# OpenAI Rate Limit Handling
MAX_REQUESTS_PER_MINUTE = 3  
requests_made = 0  

# Function to fetch news from BBC
def fetch_news():
    url = "https://www.bbc.com/business"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    for item in soup.select('[data-testid="internal-link"]'):
        title_tag = item.select_one('[data-testid="card-headline"]')

        if title_tag:
            title = title_tag.text.strip()
            link = item.get('href')
            if link.startswith("/"):
                link = "https://www.bbc.com" + link

            article_response = requests.get(link)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            paragraphs = article_soup.find_all('p')
            article_text = " ".join([p.get_text() for p in paragraphs])

            summary = summarize_text(article_text)

            articles.append({"title": title, "link": link, "summary": summary})

    return pd.DataFrame(articles)

# Function to summarize text using OpenAI
def summarize_text(text, model="gpt-4o-mini"):
    global requests_made

    if requests_made >= MAX_REQUESTS_PER_MINUTE:
        print("Rate limit reached. Waiting for 20 seconds...")
        time.sleep(20)  
        requests_made = 0  

    try:
        text = " ".join(text.split())  

        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that summarizes text concisely."},
                {"role": "user", "content": f"Summarize this text:\n{text}"}
            ],
            temperature=0.5
        )

        requests_made += 1  
        return response["choices"][0]["message"]["content"]

    except Exception as e:
        print(f"Error generating summary: {e}")
        return "Summary unavailable"

# Function to send an email with the news summary
def send_email(news_df):
    if news_df.empty:
        print("No news found. Email not sent.")
        return

    email_body = "<h2>Latest BBC Business News</h2>"
    for _, row in news_df.iterrows():
        email_body += f"<h3>{row['title']}</h3>"
        email_body += f"<p>{row['summary']}</p>"
        email_body += f'<p><a href="{row["link"]}">Read Full Article</a></p><hr>'

    msg = MIMEMultipart()
    msg['From'] = EMAIL_SENDER
    msg['To'] = EMAIL_RECEIVER
    msg['Subject'] = "Daily BBC Business News Summary"
    msg.attach(MIMEText(email_body, 'html'))

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(EMAIL_SENDER, EMAIL_PASSWORD)
        server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")

# Run script: Fetch news, summarize, send email
news_df = fetch_news()
send_email(news_df)


Rate limit reached. Waiting for 20 seconds...
Error generating summary: Rate limit reached for gpt-4o-mini in organization org-ooHRhBBjb1BYk8uZYqu2szPt on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more.
Error generating summary: Rate limit reached for gpt-4o-mini in organization org-ooHRhBBjb1BYk8uZYqu2szPt on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more.
Error generating summary: Rate limit reached for gpt-4o-mini in organization org-ooHRhBBjb1BYk8uZYqu2szPt on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more.
Error generating summary: Rate limit reached for gpt-4o-mini in organization org-ooHRhBBjb1BYk8uZYqu2szPt on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 